In [ ]:
import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
import pandas as pd
import numpy as np
import torch.nn.functional as F

# data preprocessing

In [ ]:
transform = transforms.Compose(
    [transforms.RandomHorizontalFlip(),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])


batch_size = 64
dataset = torchvision.datasets.ImageFolder("/dataset_folder", transform=transform) 
trainset,testset = torch.utils.data.random_split(dataset, [0.8,0.2]) 

trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = [classess for classess in dataset.class_to_idx]

In [ ]:
#Check number of elements for each class
import numpy as np 
import random
from matplotlib import pyplot as plt

datahist = [row[1] for row in testloader.dataset]

bins = np.arange(-50, 100, 1) 

plt.xlim([min(datahist)-5, max(datahist)+5])

plt.hist(datahist, bins=bins, alpha=0.5)
print(classes)
plt.show()

# convolutional neural network model

In [ ]:
class Net(nn.Module):
    def __init__(self, num_classes):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)  
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(256)
        self.conv5 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=1)
        self.bn5 = nn.BatchNorm2d(512)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.LazyLinear(out_features=512)
        self.dropout = nn.Dropout(0.6)
        self.fc2 = nn.LazyLinear(out_features=num_classes)
        
    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        x = self.pool(F.relu(self.bn4(self.conv4(x))))
        x = self.pool(F.relu(self.bn5(self.conv5(x))))
        x = torch.flatten(x, 1) 
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x


num_classes = 50
net = Net(num_classes)


# parameters

In [ ]:
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler


criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001, weight_decay=0.001)

device = torch.device("cuda")
if torch.cuda.is_available(): 
    torch.cuda.manual_seed(42)
    torch.cuda.manual_seed_all(42)
    net.to(device)


# training

In [ ]:
acc1 = 0.0
acc2 = 0.0
epoch = 0
while acc2<90:
    net.train()
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    net.eval()
    print('%d epoch loss: %.3f' %(epoch+1 ,  running_loss / 2000))
    epoch+=1
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Accuracy of the network: %d %%' % (100 * correct / total))
    acc1 = acc2
    acc2 = 100 * correct / total
    if acc2>acc1:
        torch.save("model.pt", "drive/MyDrive/SSNE/Projekt3/train/model.pt")
                   
        
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

with torch.no_grad():
    for data in testloader:
        images, labels = data    
        images, labels = images.to(device), labels.to(device)
        outputs = net(images)
        _, predictions = torch.max(outputs, 1)
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1

  
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print("Accuracy for class {:5s} is: {:.1f} %".format(classname, 
                                                   accuracy))

In [ ]:
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

with torch.no_grad():
    for data in testloader:
        images, labels = data    
        images, labels = images.to(device), labels.to(device)
        outputs = net(images)
        _, predictions = torch.max(outputs, 1)
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1

  
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print("Accuracy for class {:5s} is: {:.1f} %".format(classname, 
                                                   accuracy))

# Verify data with no labels

In [ ]:
from torch.utils.data import Dataset
import os
import natsort
import PIL
class CustomDataSet(Dataset):
    def __init__(self, main_dir, transform):
        self.main_dir = main_dir
        self.transform = transform
        all_imgs = os.listdir(main_dir)
        self.total_imgs = natsort.natsorted(all_imgs)
        
    def __len__(self):
        return len(self.total_imgs)

    def __getitem__(self, idx):
        img_loc = os.path.join(self.main_dir, self.total_imgs[idx])
        image = PIL.Image.open(img_loc).convert("RGB")
        tensor_image = self.transform(image)
        return tensor_image

In [ ]:
preds_list=[]
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
dataset = CustomDataSet("/test_all",transform)
trainloader = torch.utils.data.DataLoader(dataset,shuffle=False, num_workers=2)
filenames = dataset.total_imgs

loader = torch.utils.data.DataLoader(dataset,batch_size = len(dataset))
dataiter = iter(loader)

import csv

with torch.no_grad(): 
        images = next(dataiter)
        data_inputs = images.to(device)
        preds = net(data_inputs)
        preds = preds.squeeze(dim=1)
        pred_labels=(torch.argmax(preds,dim=1).view(-1,1)).tolist()
        for i in range(len(pred_labels)):
          preds_list.append([filenames[i],pred_labels[i][0]])
        

print(preds_list)

In [ ]:
from csv import writer
with open('/predictions.csv', 'w') as f_object:
 
    writer_object = writer(f_object)
 
    for i in range(0, len(preds_list)):
      writer_object.writerow(([preds_list[i][0],preds_list[i][1]]))
f_object.close()